In [8]:
# packages

import re
from tqdm import tqdm
from ast import literal_eval
import os
import gzip
from tqdm import tqdm
from collections import Counter
import os
import numpy as np
import itertools
import re
import numpy as np
import pysam
from pysam import VariantFile
import json
from statistics import mean
import statistics
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import time
import tracemalloc


ModuleNotFoundError: No module named 'pysam'

In [6]:
import pysam

ModuleNotFoundError: No module named 'pysam'

## Step1 - Preparing alignments to realign. 
1.Trimming low quality ends. -----------  2.Trimming read which exceeds insert size threshold, STR to read ref.

In [50]:
# left alignment. chr1:1762760-1762791

# Do you know what is this cell doing ? : 1.

class realign:

    # default running def and initilization of variables.
    def __init__(self, duplicated_read, read_reference_start, read_reference_end, query_seq, query_id, qualities_, bases_, cigar_len, cigar_type, end_pos_, pos_):
        
        self.duplicated_read       = duplicated_read
        self.read_reference_start  = read_reference_start
        self.read_reference_end    = read_reference_end
        self.query_seq             = query_seq
        self.query_id              = query_id
        self.qualities_            = qualities_
        self.bases_                = bases_
        self.cigar_len             = cigar_len
        self.cigar_type            = cigar_type
        self.end_pos_              = end_pos_
        self.pos_                  = pos_
        # self.recursion             = recursion
        # self.Trimmed_low_quals_list= Trimmed_low_quals_list
    
    # 
    def TrimAlignment(self, min_read_start, max_read_stop, min_base_qual, Trim_qual_bases):

            
        ltrim     = 0
        start_pos = self.pos_

        # As we use the recursion for both "Ends trimming" & "Far away STRs trimming".
        if Trim_qual_bases == True: # when trimming "low qual bases"
            i = 1
        else: i = 0 # this is when "Trim bases which are far away"
        
        # left to right trimming.        
        while((start_pos < min_read_start) and (len(self.cigar_type) > i)): #and len(cigar_ops_) > 0) ---note: cigar len is being checked & i assume all cigar lens will be greater than 0, until it is unmapped.
            
            #check if we should stop trimming b/c the quality score is above the threshold.
            qual_above_thresh = False

            # Trim the 40bp either sides of the STR.
            if Trim_qual_bases == True:    
                if self.cigar_type[0] in ['M', '=', 'X', 'I', 'S']: # do notin if 'D', 'H', cigar_ops[ltrim] was used 
                         qual_above_thresh = (self.qualities_[ltrim] > min_base_qual)
                         
            # here we check if qual is above threshold
            if qual_above_thresh == True:
                break
             
            # here we increament the start and ltrim positions. # do notin if 'H'
            if self.cigar_type[0] in ['M', '=', 'X']:
                ltrim += 1
                start_pos += 1
            elif self.cigar_type[0] in ['D']: start_pos += 1
            elif self.cigar_type[0] in ['I', 'S']: ltrim += 1

            # removing the bases using CIGAR string.
            if self.cigar_len[0] == 1:
                self.cigar_len.remove(cigar_len[0])
                self.cigar_type.remove(cigar_type[0])
            else: self.cigar_len[0] -= 1

            # removing the base from the read sequence, ( only I, S, M can be removed, D is not present in the read.)
            

        # ***************************************************************************************************************************************************
        # Right to left trimming.
        rtrim = 0
        qual_string_len = int(len(self.qualities_)) - 1
        rtrim_index     = -rtrim-1
        end_pos         = self.end_pos_

        while((end_pos > max_read_stop) and (len(self.cigar_type) > i)):


            self.qual_above_thresh = False

            # Trim the 40bp either sides of the STR.
            if Trim_qual_bases == True:    
                #
                if  self.cigar_type[0] in ['M', '=', 'X', 'I', 'S']:
                    self.qual_above_thresh = (self.qualities_[rtrim] > min_base_qual)

            
            # here we check if qual is above threshold
            if self.qual_above_thresh == True: break

            # here we increament the start and rtrim positions.
            if self.cigar_type[0] in ['M', '=', 'X']:
                rtrim -= 1
                end_pos -= 1
            elif self.cigar_type[0] in ['D']: end_pos -= 1
            elif self.cigar_type[0] in ['I', 'S']: rtrim -= 1

            # removing the bases using CIGAR string.
            if self.cigar_len[-1] == 1:
                self.cigar_len.remove(self.cigar_len[-1])
            else: self.cigar_len[-1] -= 1
                
                        

        # 
        assert(ltrim + rtrim <= len(self.bases_)), "Checking if all bases are trimmed"
        self.bases_     = self.bases_[ltrim : len(self.bases_)+rtrim]
        self.qualities_ = self.qualities_
        length_         = ltrim - rtrim
        self.pos_       = start_pos
        self.end_pos_   = end_pos


        return self.bases_, start_pos, end_pos, self.cigar_len, self.cigar_type

    
    # Function pases the min base qual and returns list with deleted low base qualities.
    def TrimLowQualityEnds(self, min_base_qual):

        return realign.TrimAlignment(self, self.end_pos_+1, self.pos_-1, min_base_qual, True)


In [51]:
# realign the reads with left normalization concept. (AlignmentOps)
'''
Realign read to reference region using left alignment variant.
Store the new alignment info using the provided alignment reference.
Converts the bases to their upper case variants.
'''
# fetch part of the sequence "sequence 1" from position 0 to 3
from pysam import FastaFile

# Do you know what is this cell doing ? : 1.

def Generate_realign_seq(chrom, ref_start, ref_end):

    fasta = "../../../../malini/repeats/reference/Homo_sapiens_assembly38.fasta"
    sequences_object   = FastaFile(fasta)
    # chrom_size         = len(sequences_object.fetch(chrom)
    reference_sequence = sequences_object.fetch(chrom, ref_start, ref_end)

    # return
    return reference_sequence


# function which creates the candidate seqs 
# def Gen_candidate_seqs():

    
    

In [52]:
# Function to generate the Cigar string from given, refn seqn and read seqn. (Global alignment)

def Generate_cigar(reference_seq, query_seq):

    # Extract CIGAR string from the first alignment
    query_lindex  = len(query_seq) - len(query_seq.lstrip('-')) # query indexs helps us to remove "leading spaces" & "trailing spaces"
    query_rindex  = len(query_seq) - len(query_seq.rstrip('-')) 
    
    
    # appending the cigar ops in order.
    cigar_ops = []
    
    for pos in range(0+query_lindex, len(query_seq)-query_rindex):
    
        # refn and query seq each base to check its type.
        rbase = reference_seq[pos]
        qbase = query_seq[pos]
        
        # checking "Macthes": represents "M"
        if rbase == qbase :
            cigar_ops.append('M')
    
        # checking "Insertion": "I"
        elif rbase == '-':
            cigar_ops.append('I')
            
        # checking "Deletion": "D"
        elif qbase == '-':
            cigar_ops.append('D')
        
        # checking "Sub": "X"
        elif rbase != qbase:
            cigar_ops.append('X')
            
    
    # list cigar ops into joint cigar string.
    cigar_ops_values = {'M':0, 'I':0, 'D':0, 'X':0}
    cigar_type     = []
    cigar_num      = []
    
    current_ops = cigar_ops[0]
    for i,ops in enumerate(cigar_ops):
    
        # keeps adding the type ops onl if consicutive.
        if ops == current_ops:
            cigar_ops_values[ops] += 1
        
        else: # here we realised order is changed with new ops.
            cigar_type.append(current_ops) # append the current info
            cigar_num.append(cigar_ops_values[current_ops])
            cigar_ops_values.update({}.fromkeys(cigar_ops_values,0)) # its necessary to keep the track of cigar ops so we clear dict so we only update order wise.
            cigar_ops_values[ops] += 1
            current_ops = ops
            
        if ops == current_ops and len(cigar_ops)-1 == int(i):
            cigar_type.append(current_ops)
            cigar_num.append(cigar_ops_values[current_ops])
        

    # return the cigar_type and cigar_num bases.
    query_seq1 = query_seq[0+query_lindex : len(query_seq)-query_rindex]
    return [cigar_type, cigar_num, query_seq1]

In [53]:
# Function to extract the seqn from realigned result.

def extract_sequence(aln_refn, aln_query, aln_start, aln_end, region_start, region_end, seq):

    # we pass the aligned query seqn and reference sequence to function Generate Cigar.
    # test print
    CIGAR_return = Generate_cigar(aln_refn, aln_query)

    cigar_type   = CIGAR_return[0]
    cigar_num    = CIGAR_return[1]
    query_seq    = CIGAR_return[2]

    # check aln start and end enclosing the STR region.
    if aln_start >= region_start or aln_end <= region_end: return False

    align_index = 0 # index of align type
    char_index  = 0 # index of current base in current CIGAR element
    pos         = aln_start
    
    # Extract region sequence if fully spanned by align.
    reg_seq = ""
    cigar_iter = 0

    #test print
    # print("E Seq", region_start, region_end, aln_start, aln_end, pos, aln_query)
    while ( cigar_iter <= len(cigar_num)):
        if char_index == cigar_num[cigar_iter]:
            # print("char_index == cigar_num", char_index, cigar_num[cigar_iter])
            cigar_iter += 1
            char_index = 0
        
        elif pos > region_end:
            if reg_seq == "":
                seq = ""
            else:
                seq = reg_seq
            # return
            # print("pos>region_end", seq, reg_seq)
            return seq  # else it will be an infinite loop
        
        elif pos == region_end:
            # print("pos == region_end", pos, region_end)
            if cigar_type[cigar_iter] == 'I':
                reg_seq += query_seq[align_index : align_index + int(cigar_num[cigar_iter])]
                align_index += int(cigar_num[i])
                char_index = 0
                cigar_iter += 1
            else:
                if reg_seq == "":
                    seq = ""
                else:
                    seq = reg_seq
                # return
                # print("pos==region_end", seq)
                return seq # else it will be infinite loop

        elif pos >= region_start:
            num_bases = min(region_end-pos, int(cigar_num[cigar_iter])-char_index)
            # print("pos>=region_start----", region_start, pos)
            if cigar_type[cigar_iter] == 'I':
                num_bases = cigar_num[cigar_iter]
                reg_seq += query_seq[align_index : align_index + num_bases]
                # print("pos >= region_start, I", reg_seq, align_index,":",num_bases, "---------", pos)
            elif cigar_type[cigar_iter] in ['M', 'X']:
                reg_seq += query_seq[align_index : align_index + num_bases]
                # print("pos >= region_start, -------------- M X", num_bases, align_index,":",num_bases, "---------", pos, query_seq[align_index : align_index+num_bases])
                pos += num_bases
            elif cigar_type[cigar_iter] == 'D':
                pos += num_bases
                # print("D---", pos)
            else: print("Invalid CIGAR char in extract reg seq")

            align_index += num_bases
            char_index  += num_bases

        else:
            num_bases = 0
            if cigar_type[cigar_iter] == 'I':
                num_bases = cigar_num[cigar_iter] - char_index
            else:
                num_bases = min(region_start - pos, cigar_num[cigar_iter] - char_index)
                # print("else---", num_bases, region_start, pos, cigar_num[cigar_iter], char_index)
                pos += num_bases
                # print("pos increament in else--", pos)

            
            align_index += num_bases
            char_index  += num_bases

    # return
    return False
    print("Logical error in extract_sequence")

In [57]:
# Here we trim identical regions.

def trim(ideal_min_length, region_start, region_end, sorted_sequences_list):

    min_len = int(len(sorted_sequences_list[1]))
    assert(min_len >= ideal_min_length), "min allele len is <= ideal_min_length"

    # removing identical haplotype sequences
    
        
        

    


In [58]:
# Function : Main function for this - add_haplotype_block() -link- get_aln_bounds() -link- gen_candidae_seqs().  

# Function to extract candidate STR sequences - using some padding to ensure indels near STR ends are included.
#                                              1.Ensure that we don't exceed the chr boundaries
#                                              2.Extract the alignment boundaries
#                                              3.Ensure that we have some spanning alignments to work with
#                                              4.Extend each VCF allele by padding size
#                                              5.Extract condidate STR seqn (using some padding to ensure indels near STR ends are included)
#                                              6.Ensure that the new haplotype block won't overlap with previous blocks
#                                              7.Add the new haplotype block

#
MIN_FRAC_READS           =  0.05
MIN_FRAC_SAMPLES         =  0.05
MIN_FRAC_STRONG_SAMPLE   =  0.2
MIN_READS_STRONG_SAMPLE  =  2
MIN_STRONG_SAMPLES       =  1
MIN_BLOCK_SPACING        =  10
REF_FLANK_LEN            =  35

list_start_end_cords1 = []


def get_aln_bounds(list_start_end_cords):

    max_aln_stop   =  2147483647
    min_aln_start  = -2147483647

    list_start_end_cords1 = list_start_end_cords
    # determining the min and max alignment boundaries.
    for i in range(int(len(list_start_end_cords)/4)):
        
        min_aln_start = min(min_aln_start, list_start_end_cords[i*4])
        max_aln_stop  = max(max_aln_stop, list_start_end_cords[i*4+1])
 
    # in pool of reads, we track the min_start and max_end of reads.
    return [min_aln_start, max_aln_stop]
    


# gen_candidate_seq checks for the valid haplotyes by referring the num of reads and samples.
def gen_candidate_seq(ref_seq, alignments_start_end, ideal_min_length, list_start_end_cords, region_start, region_end):

    sequences = []
    assert(len(sequences) == 0), "sequence list is not empty"

    sample_counts = {} # [seq][count]
    read_counts   = [] # [seq][count]
    must_inc      = []
    total_reads   = 0
    total_samples = 0

    counts = []
    sample_reads = 0
    ref_index = -1
    # Determines the number of reads and number of samples spporting each allele
    for i in range(int(len(alignments_start_end)/4)):
        
        # print(alignments_start_end[i*4+3], alignments_start_end[i*4+2], len(alignments_start_end))
        aln_start = alignments_start_end[i*4]
        aln_end   = alignments_start_end[i*4+1]
        subseq    = ""
        # print("Es--", alignments_start_end[i*4+2])
        Extracted_sequence = extract_sequence(alignments_start_end[i*4+3], alignments_start_end[i*4+2], aln_start, aln_end, region_start, region_end, subseq)
        # print("subseq--", Extracted_sequence)
        
        if Extracted_sequence != False:

            read_counts.append(Extracted_sequence)
            counts.append(Extracted_sequence)
            total_reads  += 1
            sample_reads += 1


    # Identify alleles strongly supported by sample
    read_counts_dict = Counter(read_counts)    
    counts_dict = Counter(counts)

    for seqn in counts_dict: # keys as Extraced seqn and its freqn as values.
        
        # criteria 1
        if counts_dict[seqn] >= MIN_READS_STRONG_SAMPLE and counts_dict[seqn] >= MIN_FRAC_STRONG_SAMPLE * sample_reads :
            must_inc.append(seqn)
        
        # criteria 2        
        if seqn in sample_counts:
            sample_counts[seqn] += counts_dict[seqn] * 1.0 / sample_reads
        else: 
            sample_counts[seqn] = 0
            sample_counts[seqn] += counts_dict[seqn] * 1.0 / sample_reads

    
    #
    if sample_reads > 0:
        total_samples += 1

    # Add alleles with strong support from a subset of samples.
    must_inc_dict = Counter(must_inc)
    for sqn in must_inc_dict:

        if must_inc_dict[sqn] >= MIN_STRONG_SAMPLES:
            sequences.append(sqn)
            del sample_counts[sqn]
            del read_counts_dict[sqn]
            if sqn == ref_seq:
                ref_index = len(sequences) - 1

    
    # Identify additional alleles satisfying thresholds
    for sqn in sample_counts:

        if sample_counts[sqn] > MIN_FRAC_SAMPLES*total_samples or read_counts_dict[sqn] > MIN_FRAC_READS*total_reads:
            sequences.append(sqn)
            if ref_index == -1 and sqn == ref_seq:
                ref_index = len(sequences) - 1

    # Arrange reference sequence as first element
    if ref_index == -1:
        sequences.insert(0, ref_seq)
    else:
        sequences.insert(ref_index, sequences[0])
        sequences.insert(0, ref_seq)
        

    # Sort regions by length and then by sequence, note: currently only sorting by the len.
    sorted_sequences = sorted(sequences[1:], key=lambda x: len(x))
    
    sorted_sequences_list = sequences[0:1]
    sorted_sequences_list.extend(sorted_sequences)

    # Clip identical regions
    # print("sequences--:", sorted_sequences_list)
    trim(ideal_min_length, region_start, region_end, sorted_sequences_list)
   





def add_haplotype_block(add_haplotype_block_values, alignments_start_end, start, end, LEFT_PAD, RIGHT_PAD, REF_FLANK_LEN):

    # check 1:
    # check if haplotype is passed the criteria, from the function build_haplotype()
    # code

    # check 2:
    # Ensure that we don't exceed the chromosome bounds
    chrom_size = 285000000 #temporarily using chr1 value for one region.

    if (start < REF_FLANK_LEN + LEFT_PAD   or   end + REF_FLANK_LEN + RIGHT_PAD > chrom_size):
        print("Haplotype blocks are too near to the chromosome ends")
        return False

    min_aln_start = add_haplotype_block_values[0]
    max_aln_stop  = add_haplotype_block_values[1]
    # Ensuring that we have some spanning alignmnets to work with
    region_start  = start - LEFT_PAD
    region_end    = end   + RIGHT_PAD
    refn_seq      = Generate_realign_seq("chr1", region_start, region_end) # check and pull the req seq from already pulled seq, up and down 74bp
    if (min_aln_start + 5 >= region_start or max_aln_stop - 5 <= region_end):
        print("No spanning alignment")
        return False

    # Extend each vcf allele by padding size, no idea why is this snipet being used ******* why
    # padded_vcf_alleles = []


    # Extract candidate STR seqn ( using padding to ensure indels near STR ends are included )
    ideal_min_length = 3*5 # test case
    gen_candidate_seq(refn_seq, alignments_start_end, ideal_min_length, add_haplotype_block_values, region_start, region_end)
    
     

In [59]:
# parsing the .bam file and fetching the reads corresponding to the repeat regions and getting their informative reads.

# Do you know what is this cell doing ? : 1.

Pysam_Read_directory = "../../../../malini/repeats/Short_Read_Genotyping/" 
single_Bamfile_id       = "LK66312722JN"

samfile = pysam.AlignmentFile(f'{Pysam_Read_directory}{single_Bamfile_id}.bam', "rc")

#
chrom      = "chr1" 
start      = 1762760 - 1 
end        = 1762791
repeat_len = int(end)-int(start)
LEFT_PAD       = 5
RIGHT_PAD      = 5
REF_FLANK_LEN  = 35

# Print the alignments and their scores.
alignments_start_end = []


for read in samfile.fetch(chrom,start, end):

    # 
    if read.is_supplementary or read.is_duplicate : continue
    if read.reference_start > start or read.reference_end < end: continue # @@@@@@@@ check if this line is required

    # query : start, end, min_base_quality. 
    query_start      = read.reference_start
    query_end        = read.reference_end
    min_base_qual    = 12 # ~ = 12
    
    # cigar string, (type & len)
    cigar_string = read.cigarstring
    cigar_len_t  = re.findall(r'[0-9]+', cigar_string)
    cigar_len    = []
    temp         = [cigar_len.append(int(x)) for x in cigar_len_t] 
    cigar_type   = re.findall(r'[a-zA-Z]+', cigar_string)

    # # Ignoring whos cigars only have, 'H' or 'S' with 'M'.
    # if cigar_type.count('H') != 0 or cigar_type.count('S') != 0:
    #     if cigar_type.count('I') == 0 or cigar_type.count('D') == 0 or cigar_type.count('')
    
    # qualities_, bases_ and list of cigar operations
    qualities_ = list(read.query_qualities)
    bases_     = list(read.query_sequence)

    
    # Class object, obj
    # tp
    obj = realign(read.is_duplicate, read.reference_start, read.reference_end, read.seq, read.query_name, qualities_, bases_, cigar_len, cigar_type, query_end, query_start)
    Trimmed_ends = obj.TrimLowQualityEnds(min_base_qual)

        
    # Trimming the both repeat ends which are far away, 40bp upstream and down stream
    max_read_stop = end +  40
    if start > 40: min_read_start = start - 40
    else: min_read_start = 1


    # Trimmed too far away sequence from the STR, but threshold is arbitary.
    Trimmed_bases   = obj.TrimAlignment(min_read_start, max_read_stop, 12, False)
    Seq_realign     = "".join(Trimmed_bases[0])
    realign_start   = Trimmed_bases[1]
    realign_end     = Trimmed_bases[2]
    realign_cigar_l = Trimmed_bases[3]
    realign_cigar_t = Trimmed_bases[4] 
 
    
    # Generating ref seq. *** check for if refern position is 0 and size of the chr for ref_stop.    
    Align_window_width = 75
    ref_start = realign_start - Align_window_width - 1
    ref_stop  = realign_end   + Align_window_width

    # Select only those alignments marked as good for haplotype generation.
    success = True
    
    
    # Function to realign the seqn using left align technique.
    # 1. Global alignment from Trimmed read and Expanded STR region.
    ref_seq  = Generate_realign_seq(chrom, ref_start, ref_stop)
    if len(cigar_type) > 1:
        # Global alignment
        read_seq = Seq_realign
    
    
        # Perform global alignment using Needleman-Wunsch algorithm
        alignments = pairwise2.align.globalms(ref_seq, read_seq, 2, -2, -5, -0.125, one_alignment_only=True) #(ref_eq, read_seq, match_score, mismatch_penalty, open_gap_penalty, extend_gap_penalty)
    
        new_aligned_seq = alignments[0][1]
        alignments_start_end.extend([realign_start, realign_end, new_aligned_seq, alignments[0][0]])

    elif len(cigar_type) == 1: # cross check ***********

        # here we check the flanks of the STR region to ensure indels or sub.
        check_str_region = ref_seq[realign_start - ref_start : realign_end - ref_stop]

        # if we notice var in flanks of read, then ignore it.
        if check_str_region == Seq_realign:
            alignments_start_end.extend([realign_start, realign_end, check_str_region, ref_seq])        

        # INCLUDE CODE:**********
        # here we check whether to add "sameAsref" with flank indel/sub  or  "sameAsref" without flank indel/sub.
        # code:
        # flank-var should be more than 20% of reads.
        # if flank-var >= 20% : print("likely ambigous region")


# print("len of align-:", len(alignments_start_end)/4)

# passing the arguments as list of alignment start end.
add_haplotype_block_values = get_aln_bounds(alignments_start_end)
add_haplotype_block(add_haplotype_block_values, alignments_start_end, start, end, LEFT_PAD, RIGHT_PAD, REF_FLANK_LEN)


{'CTCAAAATGACTGAATGAATGAATGAATGACTGAATAA', 'CTCAAAATGAATGAATGAATGAATGAATGAATGAATAA', 'CTCAAAATGAATGAATGAATGAATGAATGAATGAATGAATAA', 'CTCAAAATGAATGACTGAATGAATGAATGAATGAATAA'} trim
